In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
links=pd.read_csv('links.csv')

In [23]:
links['Links']

0      https://www.imf.org/en/News/Articles/2024/10/2...
1      https://www.imf.org/en/News/Articles/2024/04/1...
2      https://www.imf.org/en/News/Articles/2023/10/1...
3      https://www.imf.org/en/News/Articles/2023/04/1...
4      https://www.imf.org/en/News/Articles/2022/10/1...
                             ...                        
168    https://www.imf.org/en/News/Articles/2015/09/2...
169    https://www.imf.org/en/News/Articles/2015/09/2...
170    https://www.imf.org/en/News/Articles/2015/09/2...
171    https://www.imf.org/en/News/Articles/2015/09/2...
172    https://www.imf.org/en/News/Articles/2015/09/2...
Name: Links, Length: 173, dtype: object

In [24]:

# Data storage
data = []

# Iterate over links
for link in links['Links']:
    try:
        # Fetch the page content
        response = requests.get(link)
        response.raise_for_status()  # Ensure the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the target <section> tag
        section = soup.find('section')
        
        if section:
            # Extract the title from <h2>
            title = section.find('h2').get_text(strip=True) if section.find('h2') else "No Title"
            
            # Extract all <p> tags and combine their text into one string
            subtext = " ".join([p.get_text(strip=True) for p in section.find_all('p')])
            
            # Add to data list
            data.append({'Title': title, 'Subtext': subtext})
        else:
            # Handle missing <section>
            data.append({'Title': "No Section Found", 'Subtext': "No Subtext Found"})
    
    except Exception as e:
        # Handle errors and continue with the next link
        print(f"Error processing {link}: {e}")
        data.append({'Title': "Error", 'Subtext': str(e)})

# Convert data into a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


                                                 Title  \
0    Intergovernmental Group of Twenty-Four on Inte...   
1    Intergovernmental Group of Twenty-Four on Inte...   
2    Intergovernmental Group of Twenty-Four on Inte...   
3    Intergovernmental Group of Twenty-Four on Inte...   
4    Intergovernmental Group of Twenty-Four on Inte...   
..                                                 ...   
168  Communiqué of the Interim Committee of the Boa...   
169                    Group of Twenty Four Communiqué   
170  Communiqué of the Ministers and Governors of t...   
171                       Interim Committee Communiqué   
172  Intergovernmental Group of Twenty-Four on Inte...   

                                               Subtext  
0    October 22, 2024 1. The G-24 expresses its dee...  
1    April 16, 2024 1. The G‑24 recognizes the prof...  
2    October 10, 2023 1.               We  express ...  
3    April 11, 2023 Ministers of the Intergovernmen...  
4    October 11, 2

In [25]:
df.to_csv("scraped_data.csv", index=False)